## 1. Package

In [ ]:
! pip install xarray zarr gcsfs fsspec
! pip install --quiet climetlab
%config InteractiveShell.ast_node_interactivity = "all"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.4/255.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 120.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

In [ ]:
from torch.utils.data import Dataset
from typing import Tuple, List
import xarray as xr
import gcsfs
import os
import pandas as pd
import numpy as np
import climetlab as cml
import kagglehub
import pickle
from google.colab import auth
import time

## 2. Load and preprocessing dataset

### Load CMA Dataset

In [ ]:
path = kagglehub.dataset_download("chriszhengao/cma-best-track-data")
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/cma-best-track-data


In [ ]:
cma_raw_data = pd.read_csv(os.path.join(path, "CMA_Best_Track_Data.csv"))

/tmp/ipython-input-1656429335.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  cma_raw_data = pd.read_csv(os.path.join(path, "CMA_Best_Track_Data.csv"))


### Load ERA5 Dataset

In [ ]:
fs = gcsfs.GCSFileSystem(token='anon')

In [ ]:
era5_path = 'gs://weatherbench2/datasets/era5/'
files = fs.ls(era5_path)
print("Available files:", files)

Available files: ['weatherbench2/datasets/era5/', 'weatherbench2/datasets/era5/1959-2022-1h-240x121_equiangular_with_poles_conservative.zarr', 'weatherbench2/datasets/era5/1959-2022-1h-360x181_equiangular_with_poles_conservative.zarr', 'weatherbench2/datasets/era5/1959-2022-6h-128x64_equiangular_conservative.zarr', 'weatherbench2/datasets/era5/1959-2022-6h-128x64_equiangular_with_poles_conservative.zarr', 'weatherbench2/datasets/era5/1959-2022-6h-1440x721.zarr', 'weatherbench2/datasets/era5/1959-2022-6h-240x121_equiangular_with_poles_conservative.zarr', 'weatherbench2/datasets/era5/1959-2022-6h-512x256_equiangular_conservative.zarr', 'weatherbench2/datasets/era5/1959-2022-6h-64x32_equiangular_conservative.zarr', 'weatherbench2/datasets/era5/1959-2022-6h-64x32_equiangular_with_poles_conservative.zarr', 'weatherbench2/datasets/era5/1959-2022-6h-64x33.zarr', 'weatherbench2/datasets/era5/1959-2022-full_37-1h-0p25deg-chunk-1.zarr-v2', 'weatherbench2/datasets/era5/1959-2022-full_37-6h-0p25de

In [ ]:
era5_file_path = 'gs://weatherbench2/datasets/era5/1959-2022-full_37-6h-0p25deg-chunk-1.zarr-v2'
data_file = '/content/cleaned_cma_best_track_data.csv'

In [ ]:
era5_raw_data = xr.open_zarr(era5_file_path)

### Preprocessing CMA

In [ ]:
df = pd.DataFrame(cma_raw_data)
df.Time = pd.to_datetime(df.Time)

In [ ]:
columns = ["Tropical Cyclone Number", "Time", "Latitude", "Longitude", "Minimum Central Pressure", "Maximum Wind Speed"]
df = df[df.Time.dt.year.isin(list(range(1980, 2024)))]
df = df[df["Time"].dt.hour % 6 == 0]
df = df[columns]
df['Year'] = df['Time'].dt.year
df['SID'] = df['Tropical Cyclone Number'].astype(str) + '-' + df['Year'].astype(str)
df = df.drop(columns=['Tropical Cyclone Number'])

In [ ]:
def drop_miss_id(df):
  count = 0
  df = df.copy()
  df['Time'] = pd.to_datetime(df['Time'])
  sids = df["SID"].unique()
  drop_sids = []
  for sid in sids:
      sub_df = df[df["SID"] == sid].sort_values('Time')
      if len(sub_df) < 2:
          continue
      time_deltas = sub_df['Time'].diff().dropna()
      if not all(time_deltas == pd.Timedelta(hours=6)):
          drop_sids.append(sid)
  return drop_sids


drop_sids = drop_miss_id(df)
cma = df[~df.SID.isin(drop_sids)]

In [ ]:
df

### Preprocessing ERA5

In [ ]:
# CẤU HÌNH
LONG2SHORT_DICT = {
    "geopotential": "z", "temperature": "t", "specific_humidity": "q", "relative_humidity": "r",
    "u_component_of_wind": "u", "v_component_of_wind": "v", "vorticity": "vo", "potential_vorticity": "pv",
    "2m_temperature": "t2m", "10m_u_component_of_wind": "u10", "10m_v_component_of_wind": "v10",
    "total_cloud_cover": "tcc", "total_precipitation": "tp", "toa_incident_solar_radiation": "tisr",
    "mean_sea_level_pressure": "msl"
}

SINGLE_LEVEL_VARS = [
    "10m_u_component_of_wind", "10m_v_component_of_wind", "2m_temperature", "mean_sea_level_pressure"
] # u10, v10, t2m, tsm

MULTI_LEVEL_VARS = [
    "geopotential", "specific_humidity", "u_component_of_wind", "v_component_of_wind", "temperature"
] # z, q, u, v, t

# THAM SỐ
# option khác: height level lấy hết 37 level (trích patch 2 hoặc 3 thay vì 31 hoặc 40)
HEIGHT_LEVELS = [250, 500, 750, 1000] # các mức áp suất
TRAIN_BEGIN_YEAR = 1980
TIME_FREQUENCY = [0, 6, 12, 18] # 6h /1 track
def filter_era5_data(ds):
    """
    lọc lấy data cần thiết
    """

    datasets_to_merge = []

    # lọc single-var
    ds_single = ds[SINGLE_LEVEL_VARS]
    ds_single = ds_single.sel(time=slice(f'{TRAIN_BEGIN_YEAR}-01-01', None))
    ds_single = ds_single.sel(time=ds_single.time.dt.hour.isin(TIME_FREQUENCY))
    datasets_to_merge.append(ds_single)

    # lọc multi-var
    ds_multi = ds[MULTI_LEVEL_VARS]
    ds_multi = ds_multi.sel(time=slice(f'{TRAIN_BEGIN_YEAR}-01-01', None))
    ds_multi = ds_multi.sel(time=ds_multi.time.dt.hour.isin(TIME_FREQUENCY))
    ds_multi = ds_multi.sel(level=HEIGHT_LEVELS)
    datasets_to_merge.append(ds_multi)

    # merge
    ds_filtered = xr.merge(datasets_to_merge)

    # rename
    rename_dict = {k: v for k, v in LONG2SHORT_DICT.items() if k in ds_filtered.data_vars}
    ds_filtered = ds_filtered.rename(rename_dict)

    print(f"  Final dataset shape: {dict(ds_filtered.dims)}")
    print(f"  Final variables: {list(ds_filtered.data_vars)}")

    return ds_filtered

In [ ]:
era5 = filter_era5_data(era5_raw_data)
era5

  Final dataset shape: {'time': 61364, 'latitude': 721, 'longitude': 1440, 'level': 4}
  Final variables: ['u10', 'v10', 't2m', 'msl', 'z', 'q', 'u', 'v', 't']


/tmp/ipython-input-4096745482.py:50: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"  Final dataset shape: {dict(ds_filtered.dims)}")


<xarray.Dataset> Size: 6TB
Dimensions:    (time: 61364, latitude: 721, longitude: 1440, level: 4)
Coordinates:
  * latitude   (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float32 6kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * time       (time) datetime64[ns] 491kB 1980-01-01 ... 2021-12-31T18:00:00
  * level      (level) int64 32B 250 500 750 1000
Data variables:
    u10        (time, latitude, longitude) float32 255GB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    v10        (time, latitude, longitude) float32 255GB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    t2m        (time, latitude, longitude) float32 255GB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    msl        (time, latitude, longitude) float32 255GB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    z          (time, level, latitude, longitude) float32 1TB dask.array<chunksize=(1, 4, 721, 1440), meta=np.ndarray>
    q          (time, level, latitude, longitude) float32 1TB dask.array<chunksize=(1, 4, 721, 1440), meta=np.ndarray>
    u          (time, level, latitude, longitude) float32 1TB dask.array<chunksize=(1, 4, 721, 1440), meta=np.ndarray>
    v          (time, level, latitude, longitude) float32 1TB dask.array<chunksize=(1, 4, 721, 1440), meta=np.ndarray>
    t          (time, level, latitude, longitude) float32 1TB dask.array<chunksize=(1, 4, 721, 1440), meta=np.ndarray>

### Extract Storm Data từ CMA và ERA5

In [ ]:
import numpy as np
import pandas as pd

def extract_storm_data(era5_data, cma_data, years, PATCH=31):
    """
    Trích xuất dữ liệu từng cơn bão theo năm chỉ định

    Args:
        era5_data: fitered era5 dataset
        cma_data: pandas DataFrame với cột 'Time','SID','Latitude','Longitude','Maximum Wind Speed','Minimum Central Pressure'
        years: int hoặc list of int, các năm cần lấy (ví dụ 2005 hoặc [2005,2010])
        PATCH: kích thước patch (mặc định 31)

    Returns:
        storm_data: dict mapping SID -> list of storm_sequence dicts
    """
    # đảm bảo years là list
    if isinstance(years, int):
        years = [years]

    save_dir = 'storm_by_years'
    os.makedirs(save_dir, exist_ok=True)

    failed_extractions = 0
    HALF_PATCH = PATCH // 2

    # rút tên biến ngắn
    single_short_names = [LONG2SHORT_DICT[var]
                          for var in SINGLE_LEVEL_VARS if var in LONG2SHORT_DICT]
    multi_short_names = [LONG2SHORT_DICT[var]
                         for var in MULTI_LEVEL_VARS if var in LONG2SHORT_DICT]

    # grid ERA5
    lat_coords = era5_data.latitude.values
    lon_coords = era5_data.longitude.values
    time_coords = era5_data.time.values


    for year in years:
        storm_data = {}
        cma_per_year = cma_data[cma_data.Year == year].copy()
        total_records = len(cma_per_year)
        print(f"===================== Bắt đầu trích xuất bão năm {year} =====================")
        print(f"Tổng records của năm {year}: {total_records}")

        lat_indices = []
        lon_indices = []
        time_indices = []
        for _, row in cma_per_year.iterrows():
            lat_indices.append(np.argmin(np.abs(lat_coords - row['Latitude'])))
            lon_indices.append(np.argmin(np.abs(lon_coords - row['Longitude'])))
            time_indices.append(np.argmin(np.abs(time_coords - np.datetime64(row['Time']))))



        for i, (idx, row) in enumerate(cma_per_year.iterrows()):
            orig_time = row['Time']
            sid = row['SID']
            if sid not in storm_data:
                storm_data[sid] = []

            try:
                vmax = row['Maximum Wind Speed']
                pmin = row['Minimum Central Pressure']

                lat_idx = lat_indices[i]
                lon_idx = lon_indices[i]
                time_idx = time_indices[i]

                era5_lat = float(lat_coords[lat_idx])
                era5_lon = float(lon_coords[lon_idx])
                era5_time = time_coords[time_idx]


                # xác định slice
                lat_slice = slice(max(lat_idx - HALF_PATCH, 0),
                                min(lat_idx + HALF_PATCH + 1, len(lat_coords)))
                lon_slice = slice(max(lon_idx - HALF_PATCH, 0),
                                min(lon_idx + HALF_PATCH + 1, len(lon_coords)))

                subset = era5_data.isel(time=time_idx, latitude=lat_slice, longitude=lon_slice)

                # trích xuất multi-level vars
                da_multi = subset[multi_short_names].to_array(dim="feature")
                patch_multi = da_multi.values

                da_single = subset[single_short_names].to_array(dim="feature")
                patch_single = da_single.values

                storm_sequence = {
                    'time': era5_time,
                    'features': {
                        'multi': patch_multi,
                        'single': patch_single
                    },
                    'targets': {
                        'center_lat': era5_lat,
                        'center_lon': era5_lon,
                        'vmax': vmax,
                        'pmin': pmin
                    },
                    'metadata': {
                        'sid': sid,
                        'orig_time': orig_time,
                        'center_lat': era5_lat,
                        'center_lon': era5_lon,
                        'patch_bounds': {
                            'lat_slice': (lat_slice.start, lat_slice.stop),
                            'lon_slice': (lon_slice.start, lon_slice.stop)
                        }
                    }
                }

                storm_data[sid].append(storm_sequence)

            except Exception as e:
                failed_extractions += 1
                if failed_extractions % 10 == 0:
                    print(f"Warning: failed at record {idx}: {e}")
                continue
        total_storms = len(storm_data)
        total_records = sum(len(v) for v in storm_data.values())
        print(f"\nTrích xuất thành công cho năm {year}:")
        print(f"  Số cơn bão: {total_storms}")
        print(f"  Tổng record: {total_records}")
        print(f"  Patch thất bại: {failed_extractions}")
        # if total_storms:
        #     first_sid = next(iter(storm_data))
        #     print(f"  Multi-level patch shape: {storm_data[first_sid][0]['features'].shape}")

        storm_path = os.path.join(save_dir, f"storm_data_{year}.pkl")
        with open(storm_path, 'wb') as f:
            pickle.dump(storm_data, f)
        print(f"Đã lưu dữ liệu năm {year} vào {storm_path}")

    return storm_data


In [ ]:
storm_data = extract_storm_data(era5, cma, list(range(1980, 2022)))

===================== Bắt đầu trích xuất bão năm 2016 =====================
Tổng records của năm 2016: 725

Trích xuất thành công cho năm 2016:
  Số cơn bão: 29
  Tổng record: 725
  Patch thất bại: 0
Đã lưu dữ liệu năm 2016 vào storm_by_years/storm_data_2016.pkl
===================== Bắt đầu trích xuất bão năm 2017 =====================
Tổng records của năm 2017: 762

Trích xuất thành công cho năm 2017:
  Số cơn bão: 29
  Tổng record: 762
  Patch thất bại: 0
Đã lưu dữ liệu năm 2017 vào storm_by_years/storm_data_2017.pkl
